# Project for extra credit in Adpro

### 5a
Install whisperai on a virtual environment and make a python function that receives a file or a youtube link and generates a text extraction. The function should also receive a model name with a default for the "small" model, which is also whisper's default. (the models here determine how much memory, gpu, etc you can use, as several models have different hardware requirements).

In [2]:
# %pip install pytube
# %pip install -U openai-whisper
# ffmpeg required: 
    # Mac: brew install ffmpeg, 
    # Linux: sudo apt-get install ffmpeg, 
    # Windows: https://www.wikihow.com/Install-FFmpeg-on-Windows

import whisper
from pytube import YouTube
from IPython.display import Markdown, display
import warnings
import os

warnings.filterwarnings("ignore")

def text_extraction(file, is_link=False, model_name="small", display_text=False):
    """
    Extracts text from an audio file or YouTube video link using the specified Whisper model.

    Arguments
    ---------
    file : str
        The path to the audio file or the YouTube video link.
    is_link : bool, optional
        Whether the input is a YouTube video link. The default is False.
    model_name : str, optional
        The name of the Whisper model to use for transcription. The default is "small".
    display_text : bool, optional
        Whether to display the extracted text. The default is False.
    
    Returns
    -------
    str
        The extracted text.
    """
    try:
        # Load the appropriate Whisper model
        model = whisper.load_model(model_name)
        
        if is_link:
            yt = YouTube(file)
            audio_stream = yt.streams.filter(only_audio=True).first()
            if not audio_stream:
                raise Exception("No audio stream found in the video.")
            
            audio_file = audio_stream.download(filename='audio.mp4')
            file = audio_file  # Update file variable to the downloaded audio file

        # Perform transcription
        result = model.transcribe(file)

        if display_text:
            display(Markdown(result["text"]))

        # Clean up the downloaded audio file if it's a link
        if is_link:
            os.remove(audio_file)

        return result["text"]

    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

# Example usage with a YouTube link
ted_talk_url = "https://www.youtube.com/watch?v=z7e7gtU3PHY"
text = text_extraction(ted_talk_url, is_link=True, model_name="small", display_text=True)

 When I first told my friends that I was doing a talk on a study method that I use, I could see the collective look of disgust that slept across their faces as they processed what I just told them. So bear with me as I firmly believe that the Pomodoro method has the power to change your life. My typical cycle of studying used to start out determined. I would come home, sit down at my desk, and do a couple of worksheets. The only problem was that productiveness only lasted for an hour as I would easily get distracted. I would usually spend a couple hours on my phone and then I would snap back into determination but find myself getting burned out once again as the minutes ticked away. I would work until I physically couldn't anymore. I'd pass out, utterly exhausted. When my rigorous course choice this year, I had made myself promise that I would be productive. I had to. I had to succeed and yet I failed to do that every single day. I struggled to stay afloat, fatigued, stressed, and strained and I snapped as a result. And quite truthfully, I was disappointed, disappointed with myself. Then one day I came across a video. It was a video telling me how to study better and I was intrigued by one specific tip, the Pomodoro method. So what is it exactly? Well you start out by deciding on a task and estimating the amount of time that it will take you. Take for instance this AP World Chapter outline. I estimate that it will take me four hours of work, give or take. When instead of thinking about the outline as four hours of work, I'm going to think about it in terms of 25 minute increments or Pomodoros. So this outline would in theory take me eight Pomodoros. The next step is to work for those 25 minutes with absolutely no distractions or you have to restart the Pomodoro. But after that hyper focused work, you get to reward yourself with a five minute break which serves to recharge and refresh you in preparation for the next Pomodoro. Four cycles of this pattern of 25, five minutes and then you get to take a long break, 15 to 30 minutes. For myself, I typically still try to stay off my phone during these breaks and make some coffee, take a short walk or when I want to feel super productive, I'll do chores. I know. Shocker. This method is actually developed in the 90s by Francisco Cirillo who named the system Pomodoro which means tomato in Italian after this 25 minute kitchen timer that he used to track his work. It is important to know that although he developed the system for a 25, five minute pattern, the Pomodoro is a fluid system. It's designed to help you and help you with your work. For myself, I stick to the traditional 25, five minute pattern when I'm doing worksheets or studying for tests but for longer and more time consuming assignments like let's say projects or essays, I choose to work for much longer increments that take shorter breaks. So here I am now. I'm still not the perfect student and I want to iterate that but the Pomodoro has changed me. It's changed the way I think and act about my work. When needed, I can spend a full day simply working as I'm just recharged and kept simulated through the whole day. With a timer constantly ticking, I find myself working quickly in order to achieve and accomplish those goals through each 25 minute increment. And quite truthfully, it just feels so much more rewarding and fulfilling. Being able to check things off after the other, watching your pile of work go down, knowing that you accomplished something that day instead of not to call you out but wasting two hours on Netflix. So now it's my turn to ask you, are you as efficient as you can me? Are you productive or does your time seem to just slip away? Do you complete your work or is it scraped together at the last minute? The Pomodoro is a fluid system designed to help you produce higher quality work in a shorter amount of time. But whatever method, I encourage you to think about your time differently. To set goals for yourself and strive to meet them. To set aside the constant distractions and focus on your task at hand. You never know how much time you really have until you start to use it and it looks like my break is over. Thank you. Thank you.

In [3]:
# store text in a .txt file
with open("transcript.txt", "w") as file:
    file.write(text)

### 5b
Make another function that receives a text file and uses gpt-3.5-turbo for summarization, but the model can also be an option:https://platform.openai.com/docs/models/overview  You can make it support only gpt-3.5-turbo and gpt-4-turbo but use only 3.5, as 4 is way too much expensive!

### 5c
The summary created should be either "main_topic", "abridged", or "descriptive" (or you can find better names). Main_topic: the output should be just one or two sentences for the summary showing the main topic of the discussion in the text. "Abridged" should be 5-10 bullet points and "descriptive" should have something like 3k to 3.5k charact limit. Don't focus on the quality of the output, just on the code framework to make the prompt taking these options into account! Otherwise what is a 30m coding exercise will turn into hours of research!

In [5]:
# %pip install langchain-openai
from langchain_openai import ChatOpenAI

def GPT_summarize(file_path, summary_style='main_topic', model='gpt-3.5-turbo', display_text=False):
    """
    Summarizes the contents of a text file using a specified OpenAI model.

    Arguments:
    ---------
    file_path : str
        Path to the text file to be summarized.
    summary_style : str, optional
        Type of summary needed: 'main_topic', 'abridged', or 'descriptive'.
        Default is 'main_topic'.
    model : str, optional
        Model to use for the AI summarization, options are 'gpt-3.5-turbo' or 'gpt-4-turbo'.
        Default is 'gpt-3.5-turbo'.

    Returns:
    --------
    str
        The summarized text.
    """
    # Read the text from the file
    with open(file_path, 'r') as file:
        text = file.read()

    # Generate the prompt based on the desired summary style
    if summary_style == 'main_topic':
        prompt = f"Summarize the main topic of this text in one or two sentences:\n\n{text}"
    elif summary_style == 'abridged':
        prompt = f"Provide an abridged summary of this text in 5-10 bullet points:\n\n{text}"
    elif summary_style == 'descriptive':
        prompt = f"Write a detailed summary of this text with a character limit of 3000 to 3500 characters:\n\n{text}"
    else:
        raise ValueError("Invalid summary style specified. Choose 'main_topic', 'abridged', or 'descriptive'.")

    # Access ChatOpenAI to summarize the text
    llm = ChatOpenAI(temperature=0.1, model = model)
    summary = llm.invoke(prompt)

    # Display the summarized text if requested
    if display_text:
        display(Markdown(summary.content))

    return summary.content

# Example usage

# main topic
print("Main Topic Summary:")
summary = GPT_summarize("transcript.txt", summary_style="main_topic", display_text=True)

# bullet points
print("\nAbridged Summary:")
summary = GPT_summarize("transcript.txt", summary_style="abridged", display_text=True)

# descriptive
print("\nDescriptive Summary:")
summary = GPT_summarize("transcript.txt", summary_style="descriptive", display_text=True)

Main Topic Summary:


The main topic of this text is the Pomodoro method, a study technique that involves working in focused 25-minute intervals followed by short breaks. The author shares their personal experience with the method and encourages readers to consider using it to improve productivity and efficiency in their own work.


Abridged Summary:


- The author introduces the Pomodoro method as a study technique that has the power to change one's life
- The author describes their previous struggles with staying focused and productive while studying
- The Pomodoro method involves working for 25 minutes with no distractions, followed by a 5-minute break
- After four cycles of work and breaks, a longer break of 15-30 minutes is taken
- The method was developed in the 90s by Francisco Cirillo and is named after a kitchen timer shaped like a tomato
- The author adapts the Pomodoro method for different types of tasks, such as shorter intervals for worksheets and longer intervals for projects
- The author reflects on how the Pomodoro method has changed their approach to work and made them more efficient and productive
- The author encourages the audience to consider their own productivity and time management habits, and to set goals and focus on tasks to achieve higher quality work in a shorter amount of time
- The author concludes by emphasizing the importance of using time effectively and getting back to work using the Pomodoro method


Descriptive Summary:


The author begins by recounting their struggle with productivity and focus while studying. They describe a familiar cycle of starting out determined, only to quickly become distracted and burned out, leading to feelings of fatigue, stress, and disappointment. Despite their best efforts, they found themselves unable to stay on top of their rigorous course load. However, everything changed when they discovered the Pomodoro method.

The Pomodoro method involves breaking tasks into 25-minute intervals, or Pomodoros, followed by short breaks. The author explains how they estimate the time needed for a task, such as studying for an AP World Chapter outline, and then break it down into Pomodoros. They emphasize the importance of working with focus during the 25-minute intervals, with no distractions allowed. After each Pomodoro, they reward themselves with a short break to recharge.

The author discusses the origins of the Pomodoro method, which was developed by Francisco Cirillo in the 1990s. They explain how the system can be adapted to different tasks, such as longer work sessions for projects or essays. The author reflects on how the Pomodoro method has transformed their approach to work, allowing them to stay focused and productive throughout the day. They find the constant ticking of the timer to be motivating, and they enjoy the sense of accomplishment that comes from completing tasks efficiently.

In conclusion, the author encourages readers to consider their own productivity and time management habits. They urge readers to set goals, eliminate distractions, and focus on their tasks in order to achieve higher quality work in less time. The author emphasizes the importance of using time effectively and making the most of each moment. They challenge readers to think about how they can improve their efficiency and productivity, and they express gratitude for the positive impact the Pomodoro method has had on their own work habits.

### 5d
Make a final function (or class) that puts everything together: receives as inputs the audio and all other options for whisper and the LLM and generates a text summary.

In [6]:
import whisper
from pytube import YouTube
from IPython.display import Markdown, display
import os
import warnings
from langchain_openai import ChatOpenAI

warnings.filterwarnings("ignore")

class AudioToSummary:
    def __init__(self, model_name_whisper='small', model_name_llm='gpt-3.5-turbo', summary_style='main_topic'):
        """
        Initialize the class with model specifications and summary style.

        Parameters:
        -----------
        model_name_whisper : str
            The name of the Whisper model to use for transcription.
        model_name_llm : str
            The name of the LLM model to use for summarization.
        summary_style : str
            The style of summary ('main_topic', 'abridged', or 'descriptive').
        """
        self.model_name_whisper = model_name_whisper
        self.model_name_llm = model_name_llm
        self.summary_style = summary_style

    def text_extraction(self, file, is_link=False):
        """
        Extracts text from an audio file or YouTube video link using the specified Whisper model.

        Parameters:
        -----------
        file : str
            The path to the audio file or the YouTube video link.
        is_link : bool
            Whether the input is a YouTube video link.

        Returns:
        --------
        str
            The extracted text.
        """
        try:
            model = whisper.load_model(self.model_name_whisper)
            if is_link:
                yt = YouTube(file)
                audio_stream = yt.streams.filter(only_audio=True).first()
                if not audio_stream:
                    raise Exception("No audio stream found in the video.")

                audio_file = audio_stream.download(filename='audio.mp4')
                file = audio_file  # Update file variable to the downloaded audio file

            result = model.transcribe(file)
            text = result["text"]

            if is_link:
                os.remove(audio_file)

            return text

        except Exception as e:
            print(f"An error occurred: {e}")
            return ""

    def summarize_text(self, text):
        """
        Summarizes the extracted text using a specified LLM.

        Parameters:
        -----------
        text : str
            The text to summarize.

        Returns:
        --------
        str
            The summarized text.
        """
        prompt_mapping = {
            'main_topic': f"Summarize the main topic of this text in one or two sentences:\n\n{text}",
            'abridged': f"Provide an abridged summary of this text in 5-10 bullet points:\n\n{text}",
            'descriptive': f"Write a detailed summary of this text with a character limit of 3000 to 3500 characters:\n\n{text}"
        }

        llm = ChatOpenAI(temperature=0.1, model=self.model_name_llm)
        summary = llm.invoke(prompt_mapping[self.summary_style])

        return summary.content

    def process_audio_to_summary(self, file, is_link=False):
        """
        Processes an audio file or YouTube link to extract text and generate a summary.

        Parameters:
        -----------
        file : str
            The path to the audio file or the YouTube video link.
        is_link : bool
            Whether the input is a YouTube video link.

        Returns:
        --------
        str
            The summarized text.
        """
        text = self.text_extraction(file, is_link)
        summary = self.summarize_text(text)
        return summary

# Usage
processor = AudioToSummary(model_name_whisper='small', model_name_llm='gpt-3.5-turbo', summary_style='main_topic')
summary = processor.process_audio_to_summary("https://www.youtube.com/watch?v=z7e7gtU3PHY", is_link=True)
print(summary)

The main topic of this text is the Pomodoro method, a study technique that involves working in focused 25-minute intervals followed by short breaks. The author shares their personal experience with the method and encourages readers to consider using it to improve productivity and efficiency in their own work.
